In [1]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
node_path = '/content/drive/MyDrive/node_classification/text_classification'

In [4]:
from torch_geometric.data import Data
import numpy as np
import torch

In [19]:
import pandas as pd
df_total = pd.read_pickle(f"{node_path}/data/total_data.pkl")
dict_edges = pd.read_pickle(f"{node_path}/data/edges_data.pkl")

In [20]:
dict_edges['edges_list'] = dict_edges['edges'].apply(lambda x: list(x))

In [21]:
df_total['split'].unique()

array(['train', 'val', 'test'], dtype=object)

In [22]:
df_train = df_total[df_total['split'] == "train"].reset_index(drop=True)
df_test = df_total[df_total['split'] == "test"].reset_index(drop=True)
df_val = df_total[df_total['split'] == "val"].reset_index(drop=True)

In [23]:
len(df_train), len(df_test), len(df_val)

(22500, 25000, 2500)

In [24]:
df_train.head(3)

,text,label,embedding,embedding_normalized,split,id
0,Fräulein Doktor is as good a demonstration as ...,1,"[0.04839959, -0.22619185, -0.022073654, -0.233...","[0.012787974, -0.059763636, -0.0058322255, -0....",train,0
1,I watched this knowing almost nothing about it...,0,"[-0.15797108, -0.0759923, -0.16048081, 0.10968...","[-0.045625944, -0.02194845, -0.046350814, 0.03...",train,1
2,I must give How She Move a near-perfect rating...,1,"[0.012421426, -0.48580205, -0.09618403, 0.0802...","[0.0034402374, -0.13454771, -0.026639123, 0.02...",train,2


# Setting up the data for pytorch geometric

---

In [37]:
x = torch.tensor(np.stack(df_total['embedding_normalized'].values))
edge_index = torch.tensor(np.stack(dict_edges['edges_list'].values),  dtype=torch.long).t().contiguous()
y = torch.tensor(df_total["label"].values)

In [38]:
data = Data(x=x, edge_index=edge_index, y=y)

In [39]:
data.num_classes = len(torch.unique(data.y))

In [40]:
train_indices = list(df_train['id'].values)
val_indices = list(df_val['id'].values)
test_indices = list(df_test['id'].values)

In [41]:
train_mask = torch.zeros(len(df_total), dtype=torch.bool)
val_mask = torch.zeros(len(df_total), dtype=torch.bool)
test_mask = torch.zeros(len(df_total), dtype=torch.bool)

In [42]:
train_mask[train_indices] = True
val_mask[val_indices] = True
test_mask[test_indices] = True

In [43]:
# Add masks to the Data object
data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask

In [44]:
print(data)

Data(x=[50000, 384], edge_index=[2, 613827], y=[50000], num_classes=2, train_mask=[50000], val_mask=[50000], test_mask=[50000])


In [45]:
torch.save(data, f"{node_path}/data/ptg_complete_data.pth")